In [5]:
import pandas as pd
import sys
import math
import imp
import pymongo as pym
import json
import numpy as np
import random
import sys
sys.path.insert(0, './lib_python/')

In [6]:
urlMongoDbLocal = "mongodb://localhost:27017";
clientLocal = pym.MongoClient(urlMongoDbLocal)

In [7]:
carSharing = clientLocal["bigDive_test"]
bookingsDB = carSharing["bookings"]
hexs = carSharing["hexs_gen"]
#citychrone = clientLocal["citychrone_analisys"]["points"]

# First example: visualize the area with more bookings

In [13]:
from shapely.geometry import shape
from shapely.geometry import mapping
from shapely.geometry import MultiPolygon
import folium
import seaborn as sns
from shapely.ops import cascaded_union
import json

shellIso = np.arange(0,500, 50).tolist()
shellIso.insert(0,-1)
shellIso.append(700)
shellIso.append(1000)
shellIso.append(2000)
colorIso = sns.color_palette(palette="RdBu", n_colors = len(shellIso))
colorIso = colorIso.as_hex()
#colorIso = list(reversed(['#a50026','#d73027','#f46d43','#fdae61','#fee090','#e0f3f8','#abd9e9','#74add1','#4575b4','#313695']));


max_ = 0
count = 0
city = "Milano"
center = hexs.find_one({"city":city})["point"]["coordinates"]
map_folium =  folium.Map(location = [ center[1], center[0]], zoom_start=11)
for p in hexs.find({"city":city}):
    hex_ = p["hex"]
    num_of_starts = bookingsDB.find({
        "origin_point":{
            "$geoWithin":{
                "$geometry":hex_
            }
        }
    })
    count += 1
    num = num_of_starts.count()
    if num > 0:
        #print(num, end="\r")
        for i, val in enumerate(shellIso):
            if num < shellIso[i]:
                break
        def style(f):
            #print(f["geometry"]["style"]["fillColor"])
            return {
                    "fillColor" : f["geometry"]["style"]["fillColor"], 
                    "color":f["geometry"]["style"]["fillColor"],
                    "weight":0.1,
                    "fillOpacity":0.6
                   }
        hex_["style"] = {"fillColor":colorIso[i]}
        #print(i, colorIso[i])
        geojson_folium_simp = folium.GeoJson(hex_, style_function=style, )
        popup = folium.Popup('num of start bookings {0}'.format(num))
        popup.add_to(geojson_folium_simp)
        geojson_folium_simp.add_to(map_folium)

        max_ = max(max_, num)
    print("max {0}, count {1}".format(max_,count), end="\r")

map_folium